# Data preparation/Chunking

In [1]:
!ls data

test.json


In [11]:
import json
import os
import spacy
import faiss
from sentence_transformers import SentenceTransformer
import re

ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
nlp = spacy.load("en_core_web_sm")

def process_file(file_path):
    with open(file_path) as f:
        data = json.load(f)
        content = data["content"]
        url = data["url"]
        doc = nlp(content)

        return [{"text": sent.text, "url": url} for sent in doc.sents]

chunks = [
    chunk
    for file in os.listdir("data")
    for chunk in process_file(os.path.join("data", file))
]

chunks = [{"id": i, **chunk} for i, chunk in enumerate(chunks)]

with open("chunks.json", "w") as f:
    json.dump(chunks, f)

# Vector store

In [ ]:
sentences = [chunk["text"] for chunk in chunks]

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

embeddings = model.encode(sentences, show_progress_bar=True)

In [ ]:
faiss_index = faiss.IndexFlatIP(model.get_sentence_embedding_dimension())
faiss_index.add(embeddings)

# Retrieval/Prompt preparation

In [ ]:
base_prompt = """You are an AI assistant. Your task is to understand the user question, and provide an answer using the provided contexts. Every answer you generate should have citations in this pattern  "Answer [position].", for example: "Earth is round [1][2].," if it's relevant.

Your answers are correct, high-quality, and written by an domain expert. If the provided context does not contain the answer, simply state, "The provided contexts does not have the answer."

User question: {}

Contexts:
{}
"""

In [ ]:
k = 50
question = "Why HackerNews is so popular?"

query_embedding = model.encode([question])
distances, indices = faiss_index.search(query_embedding, k)

context = "\n".join([f"{i}. {sentences[index]}" for i, index in enumerate(indices[0])])
prompt = f"{base_prompt.format(question, context)}"

# Answer Generation

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key = "sk-deepseek-dummy-key",    
)

try:
    response = client.chat.completions.create(
        model="deepseek-r1:8b",
        temperature=0.7,
        messages=[
            {"role": "system", "content": "You are a helpful and informative assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    print(response.choices[0].message.content)
except openai.OpenAIError as e:
    print(f"An error occurred: {e}")
    print("\nPlease ensure your Ollama server is running and the model is downloaded.")


Check OPEN AI using Deepseek R1

In [ ]:
import openai
import os

# --- New syntax for openai library v1.0.0+ ---
# Import the OpenAI client`` class
from openai import OpenAI

# Create an instance of the OpenAI client.
# This client object will handle the API calls.
# Set the base_url to the correct HTTP endpoint for Ollama.
# Note that it's 'http' not 'https'
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key = "sk-deepseek-dummy-key",    
)

# Define a sample prompt for the model.
prompt = "Explain the concept of 'lazy evaluation' in programming."

try:
    # Use the client instance to make the API call with the new syntax.
    # The method is now `client.chat.completions.create`.
    # The arguments are largely the same.
    response = client.chat.completions.create(
        model="deepseek-r1:8b",
        temperature=0.7,
        messages=[
            {"role": "system", "content": "You are a helpful and informative assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # Access the content from the response object. The syntax for this also changed
    # from a dictionary-like access to a more object-oriented one.
    print(response.choices[0].message.content)

except openai.OpenAIError as e:
    # The exception class has changed, but you can still catch it.
    print(f"An error occurred: {e}")
    print("\nPlease ensure your Ollama server is running and the model is downloaded.")


<think>
Okay, so I'm trying to understand what lazy evaluation is in programming. I've heard the term before, maybe in my studies or online articles, but I'm not entirely sure about its meaning. Let me try to break it down step by step.

First, I know that evaluation in programming usually refers to how expressions are processed and values are determined. Lazy evaluation must be a specific way of handling this. Maybe it's related to when something is evaluated or computed only when needed instead of upfront. 

I remember hearing about lazy loading in databases, where certain data isn't fetched until it's actually accessed. Could that be similar? So perhaps in programming, lazy evaluation means that a value or computation is done only when it's required, not beforehand. That would save resources by avoiding unnecessary work unless it's necessary.

Wait, so if I have a function that needs some data, and instead of loading the data right away, I wait until the function is called and then 